In [1]:
import pandas as pd
from zipfile import ZipFile
import pyarrow.parquet as pq
import gzip
import json
import pyarrow as pa
import os
import geopandas as gpd
from shapely.geometry import Point, Polygon
from shapely import from_wkt

In [2]:
COLAB = True

In [3]:
DATA_DIR = "gdrive/MyDrive/Work/quantify-news/data/" if COLAB else "data/"

COMM_AREA_PATH = DATA_DIR + "CommAreas_2025.csv"
STREET_SEG_PATH = DATA_DIR + "StreetSegments_2025.geojson"
BLOCK_COMMUNITY_PATH = DATA_DIR + "BlockCommunities.csv"

In [4]:
if COLAB:
    from google.colab import drive
    drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


# Read Data

In [5]:
comm_areas = pd.read_csv(COMM_AREA_PATH)
street_segs = gpd.read_file(STREET_SEG_PATH)

In [6]:
street_type_codes = {
    "AVE": "AVENUE",
    "BLVD": "BOULEVARD",
    "CRES": "CRESCENT COURT",
    "CT": "COURT",
    "DR": "DRIVE",
    "ER": "ENTRANCE RAMP",
    "EXPY": "EXPRESSWAY",
    "HWY": "HIGHWAY",
    "LN": "LANE",
    "PKWY": "PARK WAY",
    "PL": "PLACE",
    "ROW": "ROW",
    "SQ": "SQUARE",
    "SR": "SERVICE ROAD",
    "ST": "STREET",
    "TER": "TERRACE",
    "TOLL": "TOLL WAY",
    "VIA": "WAY",
    "WAY": "EXIT RAMP"
}


In [7]:
suf_dir_codes = {
    "N": "NORTH",
    "S": "SOUTH",
    "E": "EAST",
    "W": "WEST",
    "OP": "OVERPASS",
    "IB": "INBOUND",
    "OB": "OUTBOUND",
    "NB": "NORTHBOUND",
    "SB": "SOUTHBOUND",
    "EB": "EASTBOUND",
    "WB": "WESTBOUND"
}


# Clean

## Comm areas

In [8]:
comm_areas['geometry'] = comm_areas['the_geom'].apply(from_wkt)
assert comm_areas['AREA_NUMBE'].nunique() == len(comm_areas)
assert all(comm_areas['AREA_NUM_1'] == comm_areas['AREA_NUMBE'])
comm_areas['id'] = comm_areas['AREA_NUMBE']
comm_areas = comm_areas.filter(['id','geometry','COMMUNITY'])
comm_areas = comm_areas.rename(columns={'COMMUNITY':'community_name'})
comm_areas = gpd.GeoDataFrame(comm_areas, geometry='geometry')

## Streets

In [9]:
street_cols = ['PRE_DIR','STREET_NAM','STREET_TYP','SUF_DIR','L_F_ADD','L_T_ADD','R_F_ADD','R_T_ADD','F_CROSS','T_CROSS','LENGTH','L_ZIP','R_ZIP']
street_cols = [x.lower() for x in street_cols]
assert all([c in street_segs.columns for c in street_cols])
street_segs = street_segs.filter(street_cols + ['geometry'])

In [10]:
street_segs['from_num'] = pd.to_numeric(street_segs[['l_f_add','r_f_add']].min(axis=1), 'coerce')
street_segs['to_num'] = pd.to_numeric(street_segs[['l_t_add','r_t_add']].max(axis=1), 'coerce')
street_segs = street_segs.drop(['l_f_add','r_f_add','l_t_add','r_t_add'], axis=1)

In [11]:
street_segs['street_typ_exp'] = street_segs['street_typ'].replace(street_type_codes)
street_segs['suf_dir_exp'] = street_segs['suf_dir'].replace(suf_dir_codes)
street_segs['pre_dir_exp'] = street_segs['pre_dir'].replace(suf_dir_codes)

In [12]:
street_segs['street_typ_exp'] = street_segs['street_typ_exp'].str.title()
street_segs['suf_dir_exp'] = street_segs['suf_dir_exp'].str.title()
street_segs['pre_dir_exp'] = street_segs['pre_dir_exp'].str.title()
street_segs['street_nam'] = street_segs['street_nam'].str.title()
street_segs['street_typ'] = street_segs['street_typ'].str.title()

In [13]:
def full_name(df:pd.DataFrame, pre:str, nam:str, typ:str, suf:str) -> pd.Series:
    result = df[pre].fillna('') + " " + df[nam].fillna('') + " " + df[typ].fillna('') + " " + df[suf].fillna('')
    return result.str.strip()

street_segs['full_name1'] = street_segs.pipe(full_name, 'pre_dir','street_nam','street_typ','suf_dir')
street_segs['full_name2'] = street_segs.pipe(full_name, 'pre_dir_exp','street_nam','street_typ','suf_dir_exp')
street_segs['full_name3'] = street_segs.pipe(full_name, 'pre_dir','street_nam','street_typ_exp','suf_dir')
street_segs['full_name4'] = street_segs.pipe(full_name, 'pre_dir_exp','street_nam','street_typ_exp','suf_dir_exp')

# Create street block -> community mapping

In [14]:
street_blocks = []
for row in street_segs.itertuples():
    for block in filter(lambda x: x % 100 == 0, range(row.from_num, row.to_num + 1)):
        street_blocks.append({
            'geometry': row.geometry,
            'block_name1': str(block) + " block of " + row.full_name1,
            'block_name2': str(block) + " block of " + row.full_name2,
            'block_name3': str(block) + " block of " + row.full_name3,
            'block_name4': str(block) + " block of " + row.full_name4,
        })
street_blocks = gpd.GeoDataFrame(street_blocks, geometry='geometry')

In [15]:
street_blocks = street_blocks.sjoin(comm_areas[['community_name','geometry']],
                    how='inner',
                    predicate='intersects',
                    lsuffix='street',
                    rsuffix='comm').drop(columns=['index_comm','geometry'])

In [16]:
street_blocks = (street_blocks.melt(id_vars=['community_name'],
                   value_vars=['block_name1', 'block_name2', 'block_name3', 'block_name4'],
                   var_name='exp',
                   value_name='block_name')
                .drop(columns=['exp'])
                .drop_duplicates())

In [17]:
street_blocks[['block_name','community_name']].to_csv(BLOCK_COMMUNITY_PATH)

**TODO**:
* Redo similar logic using from and to cross streets?